# zero-shot prompting 

Replace `personal_API_key` with a valid OpenAI API key. Note that the assistant_id inside the [input excel](analysis/zeroshot.xlsx) must be accessible to this API key.

- creating one single thread per company
- the user prompt includes:
    - the ESG report & G1-3 regulation pdf file defined in (esg_report_file_path & regulations_file_path)
    - the message from the [input excel](analysis/zeroshot.xlsx)
- handling the OpenAI response: retrieving the messages from the thread run -> find the assistant's response by run ID, and role=assistant
- saving the response into the input excel in a seperate column

In [ ]:
from openai import OpenAI
import pandas as pd



def runThread(assistant_id, esg_report_file_path, regulations_file_path, prompt):
  client = OpenAI(
    api_key="personal_API_key"
  )

  assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
  )

  # Upload ESG report and regulations file to OpenAI
  esg_report_file = client.files.create(
    file=open(esg_report_file_path, "rb"), purpose="assistants"
  )
  regulations_file = client.files.create(
    file=open(regulations_file_path, "rb"), purpose="assistants"
  ) 
  
  thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content": prompt,
        # Attach the new file to the user prompt.
        "attachments": [
          { "file_id": esg_report_file.id, "tools": [{"type": "file_search"}] },
          { "file_id": regulations_file.id, "tools": [{"type": "file_search"}] }
        ],
      }
    ]
  )

  run = client.beta.threads.runs.create_and_poll(
      thread_id=thread.id,
      assistant_id=assistant.id,
  )

  threadMessages = client.beta.threads.messages.list(thread.id)

  assistantMessage = next((i for i in threadMessages if i.role == "assistant" and i.run_id == run.id), None)
  return assistantMessage.content[0].text.value

In [8]:
excel_file_path = "analysis/zeroshot.xlsx"

df = pd.read_excel(excel_file_path, sheet_name='Sheet1', keep_default_na=False)

for i in df.index:
  if df['answer'][i] == '':
      answer = runThread(
        assistant_id = df['assistant_id'][i],
        esg_report_file_path = './esg_reportings/' + df['file_name'][i],
        regulations_file_path = "G1-3-regulation.pdf",
        prompt = df['prompt'][i]
      )
      df.loc[i, 'answer'] = answer
      print(answer)

      df.to_excel(excel_file_path, index=False, sheet_name='Sheet1')
      print(df)


### Analysis of Brenntag SE ESG Report Regarding G1-3 Regulations on Corruption and Bribery Prevention and Detection

#### Prevention and Identification Measures:
- **Steps Taken:** Brenntag SE has implemented measures to prevent corruption and bribery incidents by being a member of the UN Global Compact and committing to its principles, including those related to human rights, labor standards, environmental protection, and anti-corruption. The company communicates its expectations to suppliers through its Supplier Code of Conduct, requesting active protection of human rights within their organizations. Additionally, Brenntag has a whistleblowing system for employees and external third parties to report violations, although no reports were received in the reporting period【4:4†source】.
- **Policy Communication:** The company communicates its anti-corruption and anti-bribery policies and procedures through training sessions and the Supplier Code of Conduct to suppliers, emphasizing the i